### Import Libraries

In [ ]:
import os
import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import Subset
from transformers import BertConfig, BertTokenizer, BertForPreTraining
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import ModelOutput
from transformers.models.bert.modeling_bert import BertPreTrainingHeads
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from transformers import TrainerCallback
! pip install datasets
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Optional, Tuple

dir_path = '/scratch/mmt515'


### Define the Custom Model Class with Regression Head

In [18]:
from transformers import BertPreTrainedModel, BertModel
import torch.nn as nn
from transformers.modeling_outputs import ModelOutput
from transformers.models.bert.modeling_bert import BertPreTrainingHeads
from torch.nn import CrossEntropyLoss, MSELoss
import torch
from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class BertForPreTrainingWithRegressionOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    prediction_logits: torch.FloatTensor = None
    seq_relationship_logits: torch.FloatTensor = None
    regression_logits: torch.FloatTensor = None
    masked_lm_loss: Optional[torch.FloatTensor] = None
    regression_loss: Optional[torch.FloatTensor] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

class BertForPreTrainingWithRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        # Initialize BertModel with add_pooling_layer=True
        self.bert = BertModel(config, add_pooling_layer=True)
        # MLM and NSP heads
        self.cls = BertPreTrainingHeads(config)
        # Regression head
        self.regression_head = nn.Linear(config.hidden_size, 1)
        # Initialize regression head weights separately to avoid overwriting pre-trained weights
        self.regression_head.apply(self._init_weights)
        # Initialize weights
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,  # MLM labels
        next_sentence_label=None,  # NSP labels (not used in your case)
        expression=None,  # Regression labels
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # Get outputs from BertModel
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=True,  # Ensure outputs is a ModelOutput
        )

        sequence_output = outputs.last_hidden_state  # (batch_size, seq_length, hidden_size)
        pooled_output = outputs.pooler_output       # (batch_size, hidden_size)

        # MLM and NSP predictions
        prediction_scores, seq_relationship_score = self.cls(sequence_output, pooled_output)

        # Regression prediction
        regression_logits = self.regression_head(pooled_output).squeeze(-1)  # (batch_size)

        # Initialize losses
        total_loss = None
        masked_lm_loss = None
        regression_loss = None

        # Compute MLM loss if labels are provided
        if labels is not None:
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))
            total_loss = masked_lm_loss

        # Compute regression loss if expression labels are provided
        if expression is not None:
            loss_fct = MSELoss()
            regression_loss = loss_fct(regression_logits, expression)
            total_loss = total_loss + regression_loss if total_loss is not None else regression_loss

        # Return outputs using the custom ModelOutput class
        return BertForPreTrainingWithRegressionOutput(
            loss=total_loss,
            prediction_logits=prediction_scores,
            seq_relationship_logits=seq_relationship_score,
            regression_logits=regression_logits,
            masked_lm_loss=masked_lm_loss,
            regression_loss=regression_loss,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


### Load Model and Tokenizer

In [ ]:
# Paths to tokenizer and model
model_path = f"{dir_path}/local_CodonBERT_Melina/ScCodonBERT"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Load the model using the custom class
model = BertForPreTrainingWithRegression.from_pretrained(
    model_path,
    ignore_mismatched_sizes=True,
    from_tf = True
)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("Model and tokenizer loaded successfully.")


: 

In [ ]:
print(model)

BertForPreTrainingWithRegression(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(101, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

### Load Full Data (Ground Truth DNA Sequences and Expression Values)
* Some re-naming just for fun

In [ ]:
# Load data
df = pd.read_csv(f'{dir_path}/ScCodonBERT/Sc_cDNA_pep_TE30.csv')

print("Data loaded successfully.")

# Verify and rename columns for consistency
df.rename(columns={'te30  (log2)': 'expression', 'DNA sequence': 'dna_sequence', 'Categorical TE': 'categorical_te'}, inplace=True)

# Display the first few rows to verify
print(df.head())

Data loaded successfully.
  mORF name  expression  categorical_te  \
0   YAL067C   -2.173141               1   
1   YAL063C   -3.248648               1   
2   YAL054C   -2.935977               1   
3   YAL049C    0.727998               8   
4   YAL047C   -0.159598               5   

                                        dna_sequence  \
0  ATGTATTCAATTGTTAAAGAGATTATTGTAGATCCTTACAAAAGAC...   
1  ATGTCTCTGGCACATTATTGTTTACTACTAGCCATCGTCACATTGC...   
2  ATGTCGCCCTCTGCCGTACAATCATCAAAACTAGAAGAACAGTCAA...   
3  ATGGCATCTAATCAACCTGGCAAGTGTTGCTTTGAAGGAGTTTGTC...   
4  ATGGTACGTCGATGGATTCCTAGTGGCAGGCATCTTCGCAATAATG...   

                                    Protein sequence  
0  MYSIVKEIIVDPYKRLKWGFIPVKRQVEDLPDDLNSTEIVTISNSI...  
1  MSLAHYCLLLAIVTLLGLTNVVSATTAACLPANSRKNGMNVNFYQY...  
2  MSPSAVQSSKLEEQSSEIDKLKAKMSQSAATAQQKKEHEYEHLTSV...  
3  MASNQPGKCCFEGVCHDGTPKGRREEIFGLDTYAAGSTSPKEKVIV...  
4  MVRRWIPSGRHLRNNDNTGDDDDSEFTNSMDSGMSIPSLRDSMTTR...  


### Pre-process data
* Create a TE token from the category of translation efficiency (i.e. TE+category = "TE10"
* Convert DNA to RNA
* Split RNA sequence into codons
* make a columns "sequence_tokens" that contains the TE and the codons

In [21]:
# Create TE tokens
df['TE_token'] = 'TE' + df['categorical_te'].astype(str)

# Convert DNA sequences to RNA sequences
df['rna_sequence'] = df['dna_sequence'].str.upper().str.replace('T', 'U')

print("DNA sequences converted to RNA.")

def split_into_codons(sequence):
    # Split the sequence into codons of length 3
    codons = [sequence[i:i+3] for i in range(0, len(sequence), 3) if len(sequence[i:i+3]) == 3]
    return codons

# Apply the function to create a new column with codons
df['codons'] = df['rna_sequence'].apply(split_into_codons)

print("RNA sequences split into codons.")

# Combine TE tokens with codons
df['sequence_tokens'] = df.apply(lambda row: [row['TE_token']] + row['codons'], axis=1)

print("TE tokens combined with codons to form input sequences.")



DNA sequences converted to RNA.
RNA sequences split into codons.
TE tokens combined with codons to form input sequences.


In [ ]:
print(df.head())

  mORF name  expression  categorical_te  \
0   YAL067C   -2.173141               1   
1   YAL063C   -3.248648               1   
2   YAL054C   -2.935977               1   
3   YAL049C    0.727998               8   
4   YAL047C   -0.159598               5   

                                        dna_sequence  \
0  ATGTATTCAATTGTTAAAGAGATTATTGTAGATCCTTACAAAAGAC...   
1  ATGTCTCTGGCACATTATTGTTTACTACTAGCCATCGTCACATTGC...   
2  ATGTCGCCCTCTGCCGTACAATCATCAAAACTAGAAGAACAGTCAA...   
3  ATGGCATCTAATCAACCTGGCAAGTGTTGCTTTGAAGGAGTTTGTC...   
4  ATGGTACGTCGATGGATTCCTAGTGGCAGGCATCTTCGCAATAATG...   

                                    Protein sequence TE_token  \
0  MYSIVKEIIVDPYKRLKWGFIPVKRQVEDLPDDLNSTEIVTISNSI...      TE1   
1  MSLAHYCLLLAIVTLLGLTNVVSATTAACLPANSRKNGMNVNFYQY...      TE1   
2  MSPSAVQSSKLEEQSSEIDKLKAKMSQSAATAQQKKEHEYEHLTSV...      TE1   
3  MASNQPGKCCFEGVCHDGTPKGRREEIFGLDTYAAGSTSPKEKVIV...      TE8   
4  MVRRWIPSGRHLRNNDNTGDDDDSEFTNSMDSGMSIPSLRDSMTTR...      TE5   

             

### Split Data into Train, Validation, and Test Sets

In [22]:
from sklearn.model_selection import train_test_split

# Split into training and temp datasets (temp will be split into validation and test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Split temp into validation and test datasets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")


Training samples: 4271
Validation samples: 534
Test samples: 534


### Create Hugging Face Datasets

In [23]:
# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print("Datasets created.")


Datasets created.


### Define Custom Masking Function
* Use mapping of codon to amino acid
* Define custom masking function that:
    * Tokenizes without special characters (no addition of CLS, SEP, or PAD)
    * Uses first token (always TE) as CLS token (just by it being the first one, but also by setting its loss label to -100 so that it does not contribute to the MLM
    * Truncates proteins longer than 510 amino acids (BERT only takes 512 tokens)
    * Adds SEP token at the end of the sequence
    * Adds PADs if neccesary
    * Masks the TE token with prob masking_prob_TE
    * Masks codons with probability masking_prob_codons
    * Creates label tensors for training (tensors with ground thruth of masks and PAD everywhere else)
      

In [24]:
# Define the codon to amino acid mapping (Standard Genetic Code, RNA)
codon_to_aa = {
    'UUU': 'PHE', 'UUC': 'PHE', 'UUA': 'LEU', 'UUG': 'LEU',
    'CUU': 'LEU', 'CUC': 'LEU', 'CUA': 'LEU', 'CUG': 'LEU',
    'AUU': 'ILE', 'AUC': 'ILE', 'AUA': 'ILE', 'AUG': 'MET',
    'GUU': 'VAL', 'GUC': 'VAL', 'GUA': 'VAL', 'GUG': 'VAL',
    'UCU': 'SER', 'UCC': 'SER', 'UCA': 'SER', 'UCG': 'SER',
    'CCU': 'PRO', 'CCC': 'PRO', 'CCA': 'PRO', 'CCG': 'PRO',
    'ACU': 'THR', 'ACC': 'THR', 'ACA': 'THR', 'ACG': 'THR',
    'GCU': 'ALA', 'GCC': 'ALA', 'GCA': 'ALA', 'GCG': 'ALA',
    'UAU': 'TYR', 'UAC': 'TYR', 'UAA': 'STP', 'UAG': 'STP',
    'CAU': 'HIS', 'CAC': 'HIS', 'CAA': 'GLN', 'CAG': 'GLN',
    'AAU': 'ASN', 'AAC': 'ASN', 'AAA': 'LYS', 'AAG': 'LYS',
    'GAU': 'ASP', 'GAC': 'ASP', 'GAA': 'GLU', 'GAG': 'GLU',
    'UGU': 'CYS', 'UGC': 'CYS', 'UGA': 'STP', 'UGG': 'TRP',
    'CGU': 'ARG', 'CGC': 'ARG', 'CGA': 'ARG', 'CGG': 'ARG',
    'AGU': 'SER', 'AGC': 'SER', 'AGA': 'ARG', 'AGG': 'ARG',
    'GGU': 'GLY', 'GGC': 'GLY', 'GGA': 'GLY', 'GGG': 'GLY'
}


In [25]:
def custom_masking(examples, masking_prob_codons, masking_prob_TE=0.0):
    sequences = examples['sequence_tokens']
    expressions = examples['expression']
    inputs = []
    max_length = 512
    max_seq_length = max_length - 1  # Reserve space for [SEP]

    for seq_tokens, expr in zip(sequences, expressions):
        input_seq = []
        labels_seq = []
        seq_tokens_upper = [token.upper() for token in seq_tokens]
        # The first token is the TE token
        TE_token = seq_tokens_upper[0]
        # Decide whether to mask the TE token
        if np.random.rand() < masking_prob_TE:
            # Mask the TE token by replacing it with 'TE0'
            input_seq.append('TE0')
        else:
            input_seq.append(TE_token)
        # **Always set the label for TE token to -100**
        labels_seq.append(-100)
        # Process the rest of the tokens (codons)
        for codon in seq_tokens_upper[1:]:
            if len(codon) != 3:
                input_seq.append(codon)
                labels_seq.append(-100)
                continue
            if np.random.rand() < masking_prob_codons:
                aa = codon_to_aa.get(codon, None)
                if aa and tokenizer.convert_tokens_to_ids(aa) != tokenizer.unk_token_id:
                    input_seq.append(aa)
                    codon_id = tokenizer.convert_tokens_to_ids(codon)
                    labels_seq.append(codon_id)
                else:
                    input_seq.append(codon)
                    labels_seq.append(-100)
            else:
                input_seq.append(codon)
                labels_seq.append(-100)
        # Truncate the input sequence and labels if necessary
        if len(input_seq) > max_seq_length:
            input_seq = input_seq[:max_seq_length]
            labels_seq = labels_seq[:max_seq_length]
        # Manually add the [SEP] token
        sep_token_id = tokenizer.sep_token_id
        input_ids = tokenizer.convert_tokens_to_ids(input_seq)
        input_ids.append(sep_token_id)
        labels_seq.append(-100)  # For [SEP]
        attention_mask = [1] * len(input_ids)
        # Pad sequences to max_length
        sequence_length = len(input_ids)
        if sequence_length < max_length:
            padding_length = max_length - sequence_length
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            labels_seq += [-100] * padding_length
        # Convert lists to tensors
        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        labels_tensor = torch.tensor(labels_seq)
        # Prepare the final input dictionary
        input_encoding = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels_tensor,
            'expression': torch.tensor(expr, dtype=torch.float)
        }
        inputs.append(input_encoding)
    # Convert list of dictionaries to a dictionary of lists
    batch = {}
    for key in inputs[0]:
        batch[key] = torch.stack([inp[key] for inp in inputs])
    return batch


In [26]:
# Using the first two examples for testing
mock_data = {
    'sequence_tokens': df['sequence_tokens'].tolist()[:10],
    'expression': df['expression'].tolist()[:10]
}


mock_dataset = Dataset.from_dict(mock_data)


# Now test the custom_masking function
masked_batch = custom_masking(mock_dataset, masking_prob_codons=0.1, masking_prob_TE=0)

# Print the results
for i in range(len(mock_dataset)):
    input_ids = masked_batch['input_ids'][i]
    labels = masked_batch['labels'][i]
    expression = masked_batch['expression'][i]

    # Convert IDs to tokens
    input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    label_ids = labels.numpy()
    label_tokens = []
    for label_id in label_ids:
        if label_id == -100:
            label_tokens.append('[PAD]')
        else:
            label_tokens.append(tokenizer.convert_ids_to_tokens([label_id])[0])

    print(f"\nExample {i+1}:")
    print("Input Tokens:", input_tokens)
    print("Label Tokens:", label_tokens)
    print("Expression Value:", expression.item())



Example 1:
Input Tokens: ['TE1', 'AUG', 'UAU', 'UCA', 'AUU', 'GUU', 'AAA', 'GAG', 'AUU', 'AUU', 'GUA', 'GAU', 'CCU', 'UAC', 'AAA', 'AGA', 'CUA', 'LYS', 'UGG', 'GLY', 'UUU', 'AUU', 'CCA', 'GUA', 'AAG', 'CGG', 'CAG', 'GUG', 'GAA', 'ASP', 'CUG', 'CCA', 'GAU', 'GAC', 'UUA', 'AAU', 'UCA', 'ACA', 'GAA', 'AUU', 'GUC', 'ACU', 'AUC', 'UCC', 'AAC', 'AGU', 'ILE', 'CAG', 'AGU', 'CAU', 'GAA', 'ACA', 'GCU', 'GAA', 'AAU', 'UUC', 'AUC', 'THR', 'ACU', 'THR', 'AGU', 'GAA', 'AAA', 'ASP', 'CAA', 'CUA', 'CAU', 'UUU', 'GAG', 'ACU', 'SER', 'AGC', 'UAU', 'AGU', 'GLU', 'HIS', 'AAA', 'GAC', 'AAU', 'GUG', 'AAC', 'GUU', 'ACU', 'AGA', 'AGU', 'UAU', 'GAA', 'UAU', 'AGA', 'GAU', 'GAA', 'GCC', 'ASP', 'ARG', 'CCA', 'TRP', 'UGG', 'AGA', 'UUU', 'PHE', 'GAU', 'GLU', 'GLN', 'GAG', 'UAU', 'CGG', 'AUC', 'AAU', 'GAA', 'AAG', 'GAA', 'AGA', 'UCU', 'CAC', 'AAU', 'AAA', 'UGG', 'UAU', 'SER', 'TRP', 'UUC', 'LYS', 'GLN', 'GGU', 'ACC', 'UCU', 'UUC', 'AAA', 'GAA', 'AAA', 'AAA', 'UUA', 'UUA', 'AUU', 'AAA', 'UUG', 'ASP', 'GUC', 'LEU', 

In [27]:
# Assuming df_final is your DataFrame with 'sequence_tokens' column
# Calculate the protein length (number of amino acids)
df['protein_length'] = df['sequence_tokens'].apply(lambda x: len(x) - 1)  # Subtract 1 to exclude TE token


# Number of proteins longer than 510 amino acids
num_long_proteins = (df['protein_length'] > 510).sum()

# Total number of proteins
total_proteins = len(df)

# Percentage of proteins longer than 510 amino acids
percentage_long_proteins = (num_long_proteins / total_proteins) * 100

print(f"Number of proteins longer than 510 amino acids: {num_long_proteins}")
print(f"Total number of proteins: {total_proteins}")
print(f"Percentage of proteins longer than 510 amino acids: {percentage_long_proteins:.2f}%")


Number of proteins longer than 510 amino acids: 2040
Total number of proteins: 5339
Percentage of proteins longer than 510 amino acids: 38.21%


### Define Data Collator

In [28]:
class CustomDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer, masking_prob_codons=0.15, masking_prob_TE=0.0, max_length=512):
        super().__init__(tokenizer=tokenizer, padding=False)
        self.tokenizer = tokenizer
        self.masking_prob_codons = masking_prob_codons
        self.masking_prob_TE = masking_prob_TE
        self.max_length = max_length

    def set_masking_probs(self, masking_prob_codons=None, masking_prob_TE=None):
        if masking_prob_codons is not None:
            self.masking_prob_codons = masking_prob_codons
        if masking_prob_TE is not None:
            self.masking_prob_TE = masking_prob_TE

    def __call__(self, features):
        # Extract sequences and expressions from features
        sequences = [feature['sequence_tokens'] for feature in features]
        expressions = [feature['expression'] for feature in features]
        # Prepare batch for custom_masking function
        batch = {'sequence_tokens': sequences, 'expression': expressions}
        # Apply custom masking
        masked_inputs = custom_masking(batch, self.masking_prob_codons, self.masking_prob_TE)
        return masked_inputs



### Prepare Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=f'{dir_path}/local_CodonBERT_Melina/ScCodonBERT/results',
    overwrite_output_dir=True,
    num_train_epochs=10,  # Adjust the number of epochs as needed
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir=f'{dir_path}/local_CodonBERT_Melina/ScCodonBERT/logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    save_total_limit=2,
    remove_unused_columns=False, #Very important
    label_names=["labels", "expression"], #This was sooooo important to get the custom_metrics to work
    save_safetensors=False, #Not sure why, but not having this gives an error during training related to shared weights
    # Add any additional parameters as needed
)

print("Training arguments defined.")


Training arguments defined.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Define Compute Metrics Function

In [37]:
#PLEASE CHECK: I'm calculating RMSE for the regression and perplexity for the MLM but are these the best metrics?
def compute_metrics(eval_pred):
    print("compute_metrics is called")
    logits, labels = eval_pred

    # Unpack logits
    prediction_scores = logits[0]  # MLM logits (batch_size, seq_len, vocab_size)
    regression_outputs = logits[2]  # Regression logits (batch_size,)

    # Unpack labels
    labels_mlm = labels[0]  # MLM labels (batch_size, seq_len)
    expressions = labels[1]  # Regression labels (batch_size,)

    # Convert to PyTorch tensors
    prediction_scores = torch.tensor(prediction_scores)
    labels_mlm = torch.tensor(labels_mlm)
    regression_outputs = torch.tensor(regression_outputs)
    expressions = torch.tensor(expressions)

    # Compute MLM Loss and Perplexity
    loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
    vocab_size = prediction_scores.shape[-1]
    mlm_loss = loss_fct(
        prediction_scores.view(-1, vocab_size),  # Flatten logits
        labels_mlm.view(-1)  # Flatten labels
    )
    perplexity = torch.exp(mlm_loss).item()  # Convert loss to perplexity
    # Compute Accuracy
    predicted_labels = torch.argmax(prediction_scores, dim=-1)
    correct_predictions = (predicted_labels == labels_mlm).sum().item()
    total_predictions = labels_mlm.numel()
    accuracy = correct_predictions / total_predictions
    # Compute Regression RMSE
    regression_outputs = regression_outputs.view(-1)  # Ensure correct shape
    expressions = expressions.view(-1)  # Ensure correct shape

    regression_outputs_np = regression_outputs.detach().cpu().numpy()
    expressions_np = expressions.detach().cpu().numpy()
    mse = mean_squared_error(expressions_np, regression_outputs_np)
    rmse = np.sqrt(mse)
    metrics = {'eval_perplexity': perplexity, 'eval_rmse': rmse, 'eval_accuracy':accuracy}
    # Return metrics with 'eval_' prefix for Trainer
    return metrics


### Initialize Trainer

In [38]:
# Initialize the custom Data Collator
data_collator = CustomDataCollator(
    tokenizer=tokenizer,
    masking_prob_codons=0.0,  # Initial masking probability for codons
    masking_prob_TE=0.0,       # Initial masking probability for TE tokens
    max_length=512
)

# Example of updating masking probability
#This is not used
'''
def adjust_masking_prob(trainer):
    current_epoch = trainer.state.epoch
    new_masking_prob = min(0.15 + current_epoch * 0.05, 0.5)  # Increase up to 50%
    data_collator.set_masking_prob(new_masking_prob)
'''
# Add a callback to adjust masking probability
#NEED TO IMPROVE (this is a very basic masking scheduler based on epoch, we should probably base it on performance)
class MaskingProbCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        current_epoch = int(state.epoch or 0)
        # Adjust masking probabilities based on the current epoch
        new_masking_prob_codons = min(current_epoch * 0.1, 1)  # Enabled for all
        new_masking_prob_TE = min(current_epoch * 0.02, 0)       # Disable for trainign on yeast, enable for fluorescent data
        data_collator.set_masking_probs(
            masking_prob_codons=new_masking_prob_codons,
            masking_prob_TE=new_masking_prob_TE
        )
        print(f"Epoch {current_epoch}: Masking probabilities set to codons={new_masking_prob_codons:.2f}, TE={new_masking_prob_TE:.2f}")

# Include the callback in the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    callbacks=[MaskingProbCallback],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Trainer initialized.")

Trainer initialized.


<ipython-input-38-775df3089069>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Start Training

In [39]:
# Evaluate the model before training
eval_results = trainer.evaluate(trainer.eval_dataset)

print("Evaluation results before training:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


compute_metrics is called


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Evaluation results before training:
eval_perplexity: nan
eval_rmse: 1.908410668373108
eval_accuracy: 0.0
eval_loss: nan
eval_model_preparation_time: 0.007
eval_runtime: 16.2112
eval_samples_per_second: 32.94
eval_steps_per_second: 4.133


In [40]:
trainer.train()

Epoch 0: Masking probabilities set to codons=0.00, TE=0.00


Epoch,Training Loss,Validation Loss,Perplexity,Rmse,Accuracy,Model Preparation Time
1,0.000000,nan,nan,1.130552,0.000000,0.007000
2,4.053300,4.566047,48.752125,0.825029,0.004298,0.007000
3,2.257100,2.444008,6.577800,0.748463,0.050931,0.007000
4,1.927000,1.829803,3.949092,0.675447,0.099192,0.007000
5,1.552200,1.692621,3.310457,0.703916,0.139433,0.007000
6,2.286500,1.605109,3.088480,0.691300,0.178049,0.007000
7,1.610900,1.513285,2.998774,0.644908,0.217927,0.007000
8,1.281200,1.696516,2.945655,0.785381,0.256210,0.007000
9,1.312900,1.564970,2.926046,0.701483,0.291729,0.007000
10,1.378200,1.576689,2.942482,0.705633,0.326377,0.007000


compute_metrics is called
Epoch 1: Masking probabilities set to codons=0.10, TE=0.00
compute_metrics is called
Epoch 2: Masking probabilities set to codons=0.20, TE=0.00
compute_metrics is called
Epoch 3: Masking probabilities set to codons=0.30, TE=0.00
compute_metrics is called
Epoch 4: Masking probabilities set to codons=0.40, TE=0.00
compute_metrics is called
Epoch 5: Masking probabilities set to codons=0.50, TE=0.00
compute_metrics is called
Epoch 6: Masking probabilities set to codons=0.60, TE=0.00
compute_metrics is called
Epoch 7: Masking probabilities set to codons=0.70, TE=0.00
compute_metrics is called
Epoch 8: Masking probabilities set to codons=0.80, TE=0.00
compute_metrics is called
Epoch 9: Masking probabilities set to codons=0.90, TE=0.00
compute_metrics is called


TrainOutput(global_step=5340, training_loss=2.1344891794611898, metrics={'train_runtime': 4447.4929, 'train_samples_per_second': 9.603, 'train_steps_per_second': 1.201, 'total_flos': 1.132545486692352e+16, 'train_loss': 2.1344891794611898, 'epoch': 10.0})

### Evaluate model

In [ ]:
#NEED TO CHECK
#Not sure what level of masking this cell is doing
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


compute_metrics is called
Evaluation results: {'eval_perplexity': 109.70259094238281, 'eval_rmse': 1.4294681549072266, 'eval_loss': 6.742545127868652, 'eval_runtime': 5.4583, 'eval_samples_per_second': 97.833, 'eval_steps_per_second': 12.275, 'epoch': 10.0}


### Save the Fine-Tuned Model

In [ ]:
fine_tuned_model_path = f'{dir_path}/local_CodonBERT_Melina/ScCodonBERT/TE_fine_tuned_models'
trainer.save_model(fine_tuned_model_path)
tokenizer.save_pretrained(fine_tuned_model_path)

print(f"Fine-tuned model saved to {fine_tuned_model_path}")


Fine-tuned model saved to /content/drive/MyDrive/local_CodonBERT/ScCodonBERT2/TE_fine_tuned_models


### Test the Model on New Data